<a href="https://colab.research.google.com/github/CClive01/CP540_Project_2/blob/main/Rain_Culzean_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, we mount google drive and import the required tools.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import io
import pylab as plt
import numpy as np
import plotly.express as px
from datetime import datetime, timedelta
import plotly.graph_objects as go

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Create a plot of average daily rainfall on a month-by-month basis

In [ ]:
#Creates the list of years we want to examine
years_list = list(range(2000,2017))

#Creates a list of the dataframes for each year by using a loop. Process these dataframes and add them to the list "frame"
frame = []
for Year in years_list:
  filename = f'/content/drive/MyDrive/CP540 Data/Culzean Rain 2000-2017/midas-open_uk-daily-rain-obs_dv-202107_ayrshire_00999_culzean-castle_qcv-1_{Year}.csv'
  df1 = pd.read_csv(filename, skiprows=[*range(61)], skipinitialspace=True)
  df1 = df1.drop(df1.columns[[1, 2, 3, 5, 7, 8, 10, 11, 12, 13, 14]], axis=1)
  df1 = df1.drop(df1.tail(1).index, axis = 0)
  df1['ob_date'] = pd.to_datetime(df1.ob_date, format='%Y/%m/%d %H:%M:%S')
  df1.dropna()
  df1 = df1[df1.ob_day_cnt == 1]
  df1 = df1.sort_values('ob_date', ascending=True)
  df1['timestamp'] = df1['ob_date'].dt.strftime('%m/%Y') #Modify the date column into the desired format
  frame.append(df1)

#Create a single dataframe for all years by combining the dataframes in "Frame"
Combineddf = pd.concat(frame)

#Provide a list of dates which can be used for plots and finding average irradiance. We can vary the "freq=" value to add datapoints to the graph. Note: may need to change the date format
avgpr = []
spread = pd.date_range(f'2006-01-01',f'2015-12-31', freq='M').strftime("%m/%Y").tolist()

#Find the average irradiance for each point in "Spread" and add to the list "avgq"
for i in spread:
  avgpr.append((Combineddf.loc[Combineddf['timestamp'] == i, 'prcp_amt'].mean()))

#Create the plot and format as desired
plotdf = pd.DataFrame({'spread':spread, 'Daily Average Rain Fall':avgpr})
fig = px.bar(plotdf, x='spread', y='Daily Average Rain Fall')
fig.update_layout(yaxis_title='mm Rain', xaxis_title='Date', title='Average Daily Rain')
fig.show()

Create plot of required storage volumes against collection area

In [ ]:
#Specify an initial storage tank size and a range of collection areas
Tank_size = 100  # m3
areas = [150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400] #m2

#Create an empty list to store minimum values, where the lowest volume value is added
min_volume_values = []

#Loop for the areas provided
for Area in areas:
    df2 = pd.DataFrame({'datetime': Combineddf['ob_date'],
                        'rain': Combineddf['prcp_amt']})

    df2['datetime'] = pd.to_datetime(df2.datetime, format='%Y/%m/%d %H:%M:%S')
    df2.dropna(subset=['rain'], inplace=True)

    #Loop for each precipitation value in the dataframe
    vol = [Tank_size]
    for i in range(1, len(df2)):

        # Calculate the new volume based on the previous day's value, the water consumed, and the water collected from rain. Use precipitation value in metres
        new_volume = vol[i - 1] - 0.5479 + df2['rain'].iloc[i - 1] * Area * 10**-3

        #If the new volume calculated exceeds the storage volume, the water is lost and the maximum tank volume is used instead
        if new_volume > vol[0]:
            new_volume = vol[0]
        vol.append(new_volume)

    #Find the minimum volume and add to the list of minimum values
    min_volume = min(vol)
    min_volume_values.append(min_volume)

#Calculates the storage tank size based on the minimum volume found
storage_sizes = [Tank_size - value for value in min_volume_values]

#Create a plot of storage volume against collection area
custom_colors = '#018783'
figba = px.line(x=areas, y=storage_sizes, color_discrete_sequence=[custom_colors]) #Create a figure
figba.update_layout(yaxis_title='Storage Tank Volume // m3', xaxis_title='Collection Area // m2', title='Storage Volume for Varying Collection Areas') #Add titles
figba.show()

Create a plot of the volume of stored water against time for the time period

In [ ]:
#Based on above plot, select collection area and storage volume
Vessel_vol = 28 #m3
Area = 300 #m2

#Create dataframe
df3 = pd.DataFrame({'date':Combineddf['ob_date'],
                    'precipitation':Combineddf['prcp_amt']
                    })

df3['date'] = pd.to_datetime(df3.date, format='%Y/%m/%d')
df3.dropna(subset=['precipitation'], inplace=True)

#Starting at a full vessel, calculate the volume of water stored at the end of each day accounting for collection, water use, and initial stored volume
vol = [Vessel_vol]
for i in range(1, len(df3)):

    # Calculate the new volume based on the previous day's value
    new_volume = vol[i-1] - 0.5479 + df3['precipitation'].iloc[i-1]*Area/1000
    if new_volume > vol[0]:
      new_volume = vol[0]
    vol.append(new_volume)

#Create the plot
plotdf3 = pd.DataFrame({'date':df3['date'], 'precipitation':df3['precipitation']})
figdf3 = px.line(plotdf3, x=df3['date'], y=vol, color_discrete_sequence=[custom_colors])
figdf3.update_layout(yaxis_title='m3 of Water in Storage Vessel', xaxis_title='Date', title='Water Tank Stored Volume with Time')
figdf3.show()

Calculate the tank cost using Sinnott-Towler pressure vessel cost estimation and ASME vessel thickness equation

In [ ]:
import math

#Create functions to calculate the desired values
def vessel_height(Vol):
  height = (36*Vol/math.pi)**(1/3)
  return height

def pressure(height):
  press = (9780.4*height+101325)/1000000
  return press

def thickness(pressure, height):
  thick = pressure*height/(3*(2*115*0.85-1.2*pressure))+0.002
  return thick

def material_volume(height, thickness):
  volume = math.pi*(height**2)*thickness/3
  return(volume)

def cost(volume):
  capital = -10000+600*(7980*volume)**0.6
  return(capital)

#Calculate the values and find the cost
h = vessel_height(Vessel_vol)
P = pressure(h)
th = thickness(P, h)
mv = material_volume(h, th)
ce = cost(mv)
print('The cost of the storage vessel (made in 316L Steel) is: £', f"{ce:.2f}")

The cost of the storage vessel (made in 316L Steel) is: £ 39309.11
